# Multitasking in Python

Out of the box, Python works in a **sequential** way, which means, it does **one thing at a time**, one after another. If the program has to wait for some reason (e.g. a http request) it does not do anything else than waiting.

Another important fact: Python uses one **single core** only. If you have a lot to calculate, Python does not automatically distribute the work for you on multiple cores.

If we want to do things faster by implementing multitasking in Python, we first have to solve these questions:

* is the bottleneck **CPU** bound (lot of calculations, compressing, etc.)?
* is the bottleneck **IO** bound (waiting for network, slow harddisk)?

In order to understand multitasking a bit better, we need to clarify some **general concepts**:

### Parallelism

* doing calculations in parallel instead of sequential
* using more than one CPU
* hope we reach the solution in less wallclock time
* parallelism is a concept from the **solution domain**

**Examples**

* parse many files simultaneously
* distribute a computational task over many processors or nodes

### Asynchrony
* reacting to things that will happen in future
* we do not know when these things will happen
* asynchrony is **event driven**

**Examples**

* onClick mousevents in the browser
* File change notifications
* Incoming requests to a server
* Incoming packets of data to a socket

### Concurrency
* several computations are executed concurrently – during overlapping time periods – instead of sequentially
* concurrency control: coordinate access to a shared resource
* concurrency is often a part of the **problem domain**

**Examples**
* booking system for a flight
* banking account
* database updates

## Implementation concepts

To make things more challenging, Python offers various implementations. I tried to group them:

### threading

* run several «trains of thought» on a single processor
* **pre-emptive multitasking**
* The operating system knows about each thread and can interrupt it at any time to start running a different thread
* The OS decides when to switch tasks
* implemented in the [threading](https://docs.python.org/3/library/threading.html) module, using a [Queue](https://docs.python.org/3/library/queue.html#module-queue) to tackle concurrency problems
* implemented in the [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) module, using the [ThreadPoolExecutor](https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor)
* suitable for typical IO problems

### asyncio

* run several «trains of thought» on a single processor
* **cooperative multitasking**
* the tasks must cooperate by announcing when they are ready to be switched out
* the tasks decide when to give up control
* this concept has been implemented in many languages
* implemented in Python using the [asyncio](https://docs.python.org/3/library/asyncio.html) module and the `async` and `await` syntax
* suitable for IO problems

### multiprocessing

* run several «trains of thought» at the same time, using **multiple processors**
* Python creates **new processes** – a collection of resources where the resources include memory, file handles etc
* each process runs in its own Python interpreter
* implemented in the [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) module, using a [Queue](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Queue) to tackle concurrency problems
* implemented in the [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) module, using the [ProcessPoolExecutor](https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ProcessPoolExecutor)
* suitable for CPU problems

**Links** for further reading:

* https://realpython.com/python-concurrency/#what-is-concurrency
* https://realpython.com/async-io-python/#async-io-is-not-easy

## Retrieve data from websites sequentially

In [ ]:
import requests
import time

def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

## Retrieve data from websites using many threads: `concurrent.futures`

The new `concurrent.futures` module is a convenient way to achieve threading with not too much headaches. We simply have to define the maximum amount of workers, the queue is set up automatically for us.

Try to set the workers to 1 and see what happens. Increase the workers. What do you observe?

In [ ]:
import concurrent.futures
import requests
import threading
import time

thread_local = threading.local()

def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        result = executor.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

## Threading problems with global variables

Not everything is thread safe. The following example illustrates this: it lets 10 workers in our `ThreadPoolExecutor` try to increment a global counter (twice each).
In the end, we would expect a `counter` value of `10 x 2 = 20`. But is it? Run the example below a few times (you can hit CTRL+Enter to stay in the same cell). What do you observe?

Also try adjusting `sleep(0.0001)` a bit.

In [ ]:
import concurrent.futures
from time import sleep

counter = 0

def increment_counter():
    global counter
    for _ in range(2):
        current_counter_value = counter
        sleep(0.0001) # delay between read and write
        counter = current_counter_value + 1

if __name__ == "__main__":
    fake_data = list(range(5000))
    counter = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=500) as executor:
        tasks = [executor.submit(increment_counter) for _ in range(10)]
        for task in tasks:
            task.result()
    print(counter)

## Showcase the `asyncio` module: a gentle introduction

The `asyncio` module implements the Asynchronous I/O framework that can be found in many languages (JavaScript, C++, Java, Raku, etc.). Two new language elements are introduced:

* `async`: declares a function to act as a **coroutine**, short for **cooperative routine**. It means, the function announces when it is going to finish
* `await`: a marker: wait here until the function returns, in the meantime you can continue. 

In [ ]:
import asyncio

async def worker():
    print("I am working")
    await asyncio.sleep(1)
    print("I am working some more...")
    await asyncio.sleep(2)
    print("5:30 pm! Time to go home!")

async def main():
    await worker()

**Notice:** In a normal script, you would use `asyncio.run(main())` to start the **main event loop** with our `main` method. But this throws an error:

```
 asyncio.run() cannot be called from a running event loop
```

(If you are using Python 3.6, you need a [different syntax](https://docs.python.org/3.6/library/asyncio-task.html) to start the main event loop)

In [ ]:
asyncio.run(main())

Because we are in **Jupyter**, a main event loop has already started for us, so we can use `await` directly:

In [ ]:
await main()

Not very exciting, is it? Not any different than sequential programming. But what happens if we add **second worker**?

We can achieve this with `asyncio.gather(tasks)`

In [ ]:
async def main():
    asyncio.gather(worker(), worker(), worker())

In [ ]:
await main()

While the first worker starts sleeping, the second starts working, goes to sleep, passes the thread back to worker 1, etc. 

## Showcase the `asyncio` module: doing things independently of each other

Let's look at a more real-life example: we would like to **read a book** and **check our whatsapp** at the same time:

In [ ]:
import asyncio

async def reading_book():
    print("reading page 1")
    await asyncio.sleep(4)
    print("reading page 2")
    await asyncio.sleep(4)
    print("reading page 3")
    await asyncio.sleep(4)
    print("reading page 4")
    
async def seconds():
    i = 0
    while True:
        print(f"\t{i} seconds")
        i += 1
        await asyncio.sleep(1)
        if i > 20:
            break

async def checking_whatsapp():
    print("reading new message 1")
    await asyncio.sleep(2)
    print("reading new message 2")
    await asyncio.sleep(4)
    print("reading new message 3")
    await asyncio.sleep(1)
    print("reading new message 4")
    
async def main(tasks):
    await asyncio.gather(*[task for task in tasks])

In [ ]:
await main([reading_book(), checking_whatsapp(), seconds()])

## Showcase the `asyncio` module: things depend on each other

In [ ]:
import asyncio
import random
import time

async def part1(n: int) -> str:
    i = random.randint(0, 10)
    print(f"part1({n}) sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-1"
    print(f"Returning part1({n}) == {result}.")
    return result

async def part2(n: int, arg: str) -> str:
    i = random.randint(0, 10)
    print(f"part2{n, arg} sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-2 derived from {arg}"
    print(f"Returning part2{n, arg} == {result}.")
    return result

async def chain(n: int) -> None:
    start = time.perf_counter()
    p1 = await part1(n)
    p2 = await part2(n, p1)
    end = time.perf_counter() - start
    print(f"-->Chained result{n} => {p2} (took {end:0.2f} seconds).")

async def main(*args):
    await asyncio.gather(*(chain(n) for n in args))

In [ ]:
random.seed(444)   # setting the seed to a specific value allows to reproduce randomness :)
args = [1,2,3]
await main(*args)